# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
from shapely.geometry import Point
import os
import numpy as np
import re
from textwrap import shorten
from bs4 import BeautifulSoup
import gspread

## Data read-in

In [2]:
df = pd.read_csv('Stalled Projects in South Florida - Sheet1.csv')

In [3]:
# # Drop the 'URL' column if you no longer need it
# df = df.drop(columns=['URL'])

In [4]:
# df['Description'].iloc[0]

## Geocode

In [5]:
df.columns

Index(['Project Name ', 'Address', 'Google coordinates for map', 'Developer',
       'Status', 'Caption for map', 'Reporter', 'Art for map'],
      dtype='object')

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Assuming 'Google coordinates for map' contains comma-separated latitude and longitude
# Step 1: Split the coordinates into two columns if needed
df[['latitude', 'longitude']] = df['Google coordinates for map'].str.split(', ', expand=True).astype(float)

# Step 2: Create Point geometries from latitude and longitude
df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

# Step 3: Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Step 4: Set the CRS to WGS 84 (EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Now `gdf` is a GeoDataFrame with valid geometry

/Users/afarence/opt/anaconda3/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:131: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,Project Name,Address,Google coordinates for map,Developer,Status,Caption for map,Reporter,Art for map,latitude,longitude,geometry
0,American Dream Mall,Between I-75 and the Florida Turnpike,"25.940768724223318, -80.35430834436464",Ghermezian family's Triple Five,Stalled,The 2018 rezoning of the property allows Tripl...,KK,NaN,25.940769,-80.354308,POINT (-80.35431 25.94077)
1,Metropica,1800 Northwest 136th Street in Sunrise,"26.15006299314875, -80.32921473437446",Joseph Kavana's Metropica Lands LLC,Stalled,"The 65-acre mixed-use, master-planned communit...",KK,NaN,26.150063,-80.329215,POINT (-80.32921 26.15006)
2,Magic City Innovation District,Between Northeast 60th Street and 64th Street ...,"25.832092902164383, -80.1903505634395",The master developers are Plaza Equity Partner...,Slow moving,The Magic City Innovation District is a roughl...,Lidia,Pls. see link in Art section of Google Doc,25.832093,-80.190351,POINT (-80.19035 25.83209)
3,Miami Produce Center,"2140 Northwest 12th Avenue, Miami FL","25.797111205113104, -80.21650974708585",Robert Wennett,Stalled,Developer Robert Wennett scored approval in 20...,Lidia,attached on Monday.com,25.797111,-80.216510,POINT (-80.21651 25.79711)
4,"One River Point, now Faena Residences",24 Southwest Fourth Street in Miami,"25.769806330026157, -80.19419873439219",Shahab Karmely's KAR Properties and Edgardo De...,Revived,"Shahab Karmely, then backed by hedge fund mana...",KK,NaN,25.769806,-80.194199,POINT (-80.19420 25.76981)
5,The River District,242 Southwest Fifth Street in Miami\n501 South...,"25.769622274497042, -80.1987194",Chetrit Group,Revived,"In 2015, the Miami City Commission approved a ...",KK,NaN,25.769622,-80.198719,POINT (-80.19872 25.76962)
6,Moishe Mana's downtown Miami district,NaN,"25.77410091137202, -80.19084997145112",NaN,Slow moving,It's been more than a decade since Moishe Mana...,Lidia,Attached on Monday.com,25.774101,-80.190850,POINT (-80.19085 25.77410)
7,Moishe Mana's Wynwood project,NaN,"25.799384787383566, -80.20388432958792",NaN,Slow moving,Moishe Mana's Wynwood special area plan is amo...,Lidia,Attached on Monday.com,25.799385,-80.203884,POINT (-80.20388 25.79938)
8,Sabal Palm Village at Flagler Trail,NaN,"25.823925571753556, -80.1894693873888",NaN,Stalled,"SPV Realty, tied to New Yorker Justin Podolsky...",Lidia,attached on Monday.com,25.823926,-80.189469,POINT (-80.18947 25.82393)
9,Miami Beach Convention Center,NaN,"25.79263962496994, -80.1344550831726",NaN,Revived,Miami Beach voters approved a referendum for t...,Lidia,attached on Monday.com,25.792640,-80.134455,POINT (-80.13446 25.79264)


In [7]:
gdf.columns

Index(['Project Name ', 'Address', 'Google coordinates for map', 'Developer',
       'Status', 'Caption for map', 'Reporter', 'Art for map', 'latitude',
       'longitude', 'geometry'],
      dtype='object')

In [8]:
gdf = gdf.rename(columns={'Caption for map':'Details'})

In [9]:
# Strip whitespace from all string columns
gdf.columns = gdf.columns.str.strip()  # Trim whitespace from column names
gdf = gdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace from string values

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_72902/2323350875.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gdf = gdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Trim whitespace from string values


In [10]:
# Convert all data to strings if possible, or remove complex structures
for col in gdf.columns:
    if gdf[col].dtype == object:
        try:
            gdf[col] = gdf[col].astype(str)
        except Exception as e:
            print(f"Could not convert column {col}: {e}")
            gdf.drop(columns=[col], inplace=True)


In [11]:
gdf.to_file('data.geojson', driver='GeoJSON')

In [12]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFla_Stalled_Projects


In [13]:
gdf.columns

Index(['Project Name', 'Address', 'Google coordinates for map', 'Developer',
       'Status', 'Details', 'Reporter', 'Art for map', 'latitude', 'longitude',
       'geometry'],
      dtype='object')

<a href="https://therealdeal.com/chicago/2024/04/03/david-gassman-sells-chicago-apartment-portfolio-for-72m/"> 6954 North Sheridan Road, Chicago</a>